<a href="https://colab.research.google.com/github/digitalhumanitiestextbook/dhtextbook/blob/main/chapter09/9_topic_modeling_practice.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# 패키지 설치
!pip uninstall numpy scipy gensim pandas -y
!pip install numpy==1.26.4 -qq
!pip install scipy==1.11.4 -qq
!pip install pandas==2.2.2 -qq
!pip install gensim==4.3.2 -qq

In [ ]:
# 세션 재실행
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
# 패키지 설치
!pip install pyLDAvis -qq
!pip install -qq -U gensim
!pip uninstall spacy thinc -y
!pip install spacy==3.6.1 --no-deps -qq
!pip install thinc==8.1.12 --no-deps -qq
!pip install matplotlib -qq
!pip install seaborn -qq
!python -m spacy download en_core_web_md -qq

- 아래 코드를 실행한 후 나타나는 "파일 선택" 버튼을 클릭하여, 분석할 txt 파일을 업로드하세요.

In [ ]:
from google.colab import files
import os

# 파일 업로드
uploaded = files.upload()

# 업로드한 파일 경로 가져오기
file_path = list(uploaded.keys())[0]
root, ext = os.path.splitext(file_path)

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd

# NLTK의 영어 문장 토큰화 도구 다운로드
nltk.download('punkt')
nltk.download('punkt_tab')

# 파일 읽기
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

# 문장 단위 분리
texts = sent_tokenize(text)

# 데이터프레임 초기화
data = pd.DataFrame({
    "number": range(1, len(texts) + 1),
    "text": texts
})

data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import pyLDAvis.gensim_models
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

sns.set()  # Seaborn 기본 스타일 적용
pyLDAvis.enable_notebook()  # PyLDAvis가 주피터 노트북 내부에 바로 시각화되도록 설정

In [ ]:
# spaCy 모델:
nlp = en_core_web_md.load()

# 글에서 제거하고 싶은 품사태그 입력 (부사(ADV), 대명사(PRON), 접속사(CCONJ), 구두점(PUNCT), 입자(PART), 한정사(DET), 전치사/후치사(ADP), 공백(SPACE), 수사(NUM), 기호(SYM), 동사(VERB), 형용사(ADJ))
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM', 'VERB', 'ADJ']

# 불용어 사용자 설정
custom_stop_words = ['nan', 'nat', 'shall', 'oh', 'mrs', 'mr', 'ms', 's', 'yes', 'shd', 'cd', 'wh', 'wd'] # 사용자가 임의로 지정한 불용어 추가하기
tokens = []

# 'text'열을 스트링으로 전환하여 잠재적으로 누락될 수 있는 값 처리
for text in nlp.pipe(data['text'].astype(str).fillna('')):
   proj_tok = [token.lemma_.lower() for token in text
               if token.pos_ not in removal and
               not token.is_stop and
               token.is_alpha and
               token.lemma_.lower() not in custom_stop_words]
   tokens.append(proj_tok)

In [ ]:
# 토큰화된 결과를 데이터프레임에 추가
data['tokens'] = tokens
data['tokens']

In [ ]:
# 딕셔너리로 각 단어마다 고유 아이디 부여
dictionary = Dictionary(data['tokens'])

# 빈도 기준으로 드문 단어와 너무 흔한 단어 제거 
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

# 코퍼스 생성: 각 문서를 (단어 ID, 단어 빈도) 리스트로 변환
corpus = [dictionary.doc2bow(doc) for doc in data['tokens']]

In [ ]:
#C_umass를 이용해서 일관성 점수 구하기
topics = []  # 토픽 개수 기록
score = []  # 각 토픽 개수별 Coherence Score 기록

# 토픽 개수를 1부터 19까지 변화시키며 평가
for i in range(1, 20):
   # LDA 모델 학습
   lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=10, num_topics=i, workers = 4, passes=15, random_state=0)
   
   # CoherenceModel로 토픽 일관성 평가
   coh_model = CoherenceModel(model=lda_model, corpus=corpus, dictionary=dictionary, coherence='u_mass')

   # 토픽 개수 및 점수 저장
   topics.append(i)
   score.append(coh_model.get_coherence())

# 토픽 개수 vs Coherence Score 시각화
plt.plot(topics, score)  # 토픽 개수에 따른 Coherence Score 곡선 그리기
plt.xlabel('Number of Topics')  # x축 레이블 설정
plt.ylabel('Coherence Score')  # y축 레이블 설정
plt.show()  # 그래프 출력

In [ ]:
#c_v를 사용해서 일관성 점수 구하기
topics = []  # 토픽 개수 기록
score = []  # 각 토픽 개수별 Coherence Score 기록

# 토픽 개수를 1부터 19까지 변화시키며 평가
for i in range(1, 20):
   # LDA 모델 학습
   lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=10, num_topics=i, workers = 4, passes=15, random_state=0)

   # CoherenceModel로 토픽 일관성 평가
   coh_model = CoherenceModel(model=lda_model, texts=data['tokens'], corpus=corpus, dictionary=dictionary, coherence='c_v')
   
   # 토픽 개수 및 점수 저장
   topics.append(i)
   score.append(coh_model.get_coherence())

# 토픽 개수 vs Coherence Score 시각화
plt.plot(topics, score)  # 토픽 개수에 따른 Coherence Score 곡선 그리기
plt.xlabel('Number of Topics')  # x축 레이블 설정
plt.ylabel('Coherence Score')  # y축 레이블 설정
plt.show()  # 그래프 출력

In [ ]:
# LDA 모델 학습
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50, num_topics=10, random_state=0, passes=15)

# 학습된 모델의 모든 토픽 출력
lda_model.print_topics(-1)

In [ ]:
#시각화 생성
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

# 시각화 저장
pyLDAvis.save_html(lda_display, 'index.html')

In [ ]:
# Google Colab 환경에서 생성된 HTML 파일을 로컬로 다운로드
from google.colab import files

files.download("index.html")

In [ ]:
def make_topictable(lda_model, corpus):
    topic_table = pd.DataFrame()  # 결과를 저장할 빈 DataFrame 생성

    # 각 문서별 토픽 분포 계산
    for i, topic_dist in enumerate(lda_model[corpus]):
        # ldamodel.per_word_topics가 True면 topic_list[0]을 fasle면 topic_list 전체를 할당
        doc = topic_dist[0] if lda_model.per_word_topics else topic_dist

        # 각 문서에 대해 가장 비중이 높은 토픽 찾기
        if doc:  # 토픽 분포가 비어있지 않은 경우
            top_topic, weight = max(doc, key=lambda x: x[1]) # (토픽 ID, 확률) 중 확률이 가장 높은 것
            new_row = pd.DataFrame([pd.Series([int(top_topic), round(weight, 4), topic_dist])]) # 새 행 생성: [가장 높은 토픽 ID, 그 토픽의 비중, 전체 토픽 분포]
            topic_table = pd.concat([topic_table, new_row], ignore_index=True) # 기존 topic_table에 행 추가

    return topic_table

In [ ]:
topictable = make_topictable(lda_model, corpus)  # 토픽 테이블 생성
topictable = topictable.reset_index()  # 문서 번호를 의미하는 열로 사용하기 위해 인덱스를 추가
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']  # 각 열의 의미를 명확히 지정
topictable[:1614]  # 상위 1614개 문서의 토픽 정보 확인

In [ ]:
from google.colab import files
topictable.to_csv("topictable.csv", index=False)
files.download('topictable.csv')